In [1]:
import numpy as np
import pandas as pd
import itertools
import re
import os
import keras
import nltk

In [2]:
from pandas import DataFrame

from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.pooling import GlobalMaxPooling1D
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, GRU, Embedding, Dropout, MaxPooling1D, Conv1D, AveragePooling1D, Flatten,Bidirectional,BatchNormalization, LSTM, SpatialDropout1D
from keras.initializers import TruncatedNormal

from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sebas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def removeStopWords_Lemmatize(s):
    stopset = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(s)
    tempList = [token.lower() for token in tokens if token.lower() not in stopset and  len(token)>2]
    tempList = [lemmatizer.lemmatize(w) for w in tempList]
    return " ".join(x for x in tempList)

In [4]:
def reduce_lengthening(text):

    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [5]:
data=pd.read_csv("gumtree Scrapping data.csv")
data.head(2)

,Unnamed: 0,Title,Description,Price,Location,Ad Link,Category,Latitude,Longitude
0,0,Maple dining table with chairs (matching Side ...,Beautiful Solid Queensland fiddleback maple di...,$800 negotiable,"Bayside Area, Brighton",/s-ad/brighton/antiques/maple-dining-table-wit...,antiques-art-collectables,-37.9081962,144.9957991
1,1,Art Deco painting (from original).,Art Deco painting on board with frame from ori...,$600,"Glen Eira Area, Bentleigh",/s-ad/bentleigh/art/art-deco-painting-from-ori...,antiques-art-collectables,-37.9185101,145.0409246


In [6]:
# fill the null values with some self-explanotory text
data["Description"]=data["Description"].fillna("Undescribed")
data["Price"]=data["Price"].fillna("Unpriced")

# filter out for non characters
data["Title"]=data["Title"].apply(lambda x: re.sub(r'\W+', ' ', x))
data["Description"]=data["Description"].apply(lambda x: re.sub(r'\W+', ' ', x))
data["Price"]=data["Price"].apply(lambda x: re.sub(r'\W+', ' ', x))

In [7]:
banned = ['and', 'to', 'the', 'a', 'for','with', 'in','is', 'of', 'or', 'on', 'you', 'are', 'from', 'up', 'available'
               ,'The', 'Have', 'all', 'at', 'as', 'condition' , 's', 'I', 'your', 'can',  'our', 'new',
                'it', 'be', 'We','x', 'This','has','will','only','this', 'New', 'pick','my','an''very', 'if',
               'Please', 'also','more','no', 'but', 'Size', 'Melbourne', 'been', 'A', 'store', 'Brand', 'sale', 'any'
               ,'great','It', 'well', 'price','All', 'us', 'me', 'so', 'just', 'Australia', 't', 'Hey', "i'm", 'hey','hi'
         , 'Hi',"I’m", 'negotiable' ]

In [8]:
f = lambda x: ' '.join([item for item in x.split() if item not in banned])

In [9]:
# concatinate the Title, Description and Price column
data["X"]= data["Title"].apply(str)+" "+data["Description"].apply(str)+" "+data["Price"].apply(str)

In [10]:
out = ' '.join(data["X"].astype(str))
words = re.findall('\w+',out)

In [11]:
#Remove banned words words
data["X"] = data["X"].astype(str).apply(f)

#Remove Stop words
data["X"]=data["X"].apply(lambda x: removeStopWords_Lemmatize(x))

#Fix the extra letters like Yesssss
data["X"]= data["X"].apply(lambda x: reduce_lengthening(x))

In [12]:
x=data["X"]
targets= data["Category"].values
#targets = targets.reshape((len(targets), 1))

tokenizer = Tokenizer( filters='!"#$%&()*+,-./:;<=>@[\\]^_`{|}~\t\n', lower=True)

tokenizer.fit_on_texts(x)

vocab_size = len(tokenizer.word_index) + 1

In [13]:
x_train, x_test, y_train, y_test= tts(x,targets,test_size=0.2, random_state=4)

x_tr=tokenizer.texts_to_sequences(x_train)
x_ts=tokenizer.texts_to_sequences(x_test)

x_tr=pad_sequences(x_tr, maxlen=250)
x_ts=pad_sequences(x_ts, maxlen=250)

np.shape(y_train)

(1925,)

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

#Create the DTM first
cv=CountVectorizer(stop_words='english')
train_dtm=cv.fit_transform(x_train)
test_dtm=cv.transform(x_test)

In [15]:
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    y_test_enc = np_utils.to_categorical(y_test_enc)
    y_train_enc = np_utils.to_categorical(y_train_enc)
    return y_train_enc, y_test_enc

In [16]:
y_train, y_test=prepare_targets(y_train, y_test)
np.shape(y_train)

(1925, 12)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from skmultilearn.adapt import MLkNN

knn_classifier = KNeighborsClassifier(n_neighbors=1)
knn_classifier.fit(train_dtm, y_train)
knn_classifier.score(test_dtm, y_test)

0.5912863070539419

In [27]:
#Predict testing set
y_pred = knn_classifier.predict(test_dtm)
#Check performance using accuracy
print(accuracy_score(y_test, y_pred))
#Check performance using roc
# roc_auc_score(y_test, y_pred)


# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

0.5912863070539419


In [28]:
import sklearn.metrics as metrics

metrics.hamming_loss(y_test, y_pred)

0.06811894882434301

In [36]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=700)
rf_classifier.fit(train_dtm, y_train)
rf_classifier.score(test_dtm, y_test)

0.5892116182572614

In [30]:
from sklearn.ensemble import ExtraTreesClassifier

eT_classifier = ExtraTreesClassifier(n_estimators=500, max_features = "sqrt")
eT_classifier.fit(train_dtm, y_train)
eT_classifier.score(test_dtm, y_test)

0.6161825726141079